In [17]:
import os.path as osp
import os
import sys
sys.path.append('./laughter')
from laughter_detection import LaughterDetection
from face_extractor.face_extractor import FaceExtractor
from recognition.face_recog import FaceRecognizer
from final_shorts.final_timeline import make_final_timeline
from moviepy.editor import *

## assign file path

In [19]:
VIDEO_PATH = '/opt/ml/project/input_dir/video/testvideo_3_1.mp4'
WAV_PATH = '/opt/ml/project/input_dir/wav/testwav_3_1'
LAUGHTER_MODEL_PATH = '/opt/ml/project/model/laughter'
CLUSTER_IMG_SAVE_PATH = '/opt/ml/project/output_dir/cluster/testwav_3_1'
SHORTS_SAVE_PATH = '/opt/ml/project/output_dir/shorts/'
SHORTS_NAME = 'RadioStar_EP00_'

## laughter detection

In [3]:
shorts_timeline = LaughterDetection(video_path=VIDEO_PATH, wav_path=WAV_PATH, ml_path=LAUGHTER_MODEL_PATH)

## clustering

In [4]:
extractor = FaceExtractor(VIDEO_PATH,None,CLUSTER_IMG_SAVE_PATH,threshold=0.57,face_cnt=200)
fingerprints = extractor.extract_fingerprints()
clusters = extractor.cluster_fingerprints(fingerprints)
merged_clusters = extractor.merge_clusters(clusters,fingerprints)
final_cluster_results = extractor.get_final_dict()

--------------------------------------------------------------------------------
[Source Video File]: /opt/ml/project/input_dir/video/testvideo_3_1.mp4
[Frame resolution H x W]: (1080.0 x 1920.0)
[FPS]: 29.97
[Total number of frames]: 32391
[Total number of seconds]: 1080
[Similiarty Threshold]: 0.57
[Number of target faces for clustering]: 200
--------------------------------------------------------------------------------
>>> Extracting fingerprints...
>>> Clustering fingerprints...
>>> Merging clusters...
>>> Calculating average encoding and representative encoding...


## recognition

In [5]:
target_people = ['person_0', 'person_1']
target_encoding = [final_cluster_results[person]['repr_encoding'] for person in target_people]
recognizer = FaceRecognizer(VIDEO_PATH, target_encoding=target_encoding)
timelines, output_frames = recognizer.recognize_faces()
people_timeline = recognizer.make_people_timeline(timelines, output_frames)

## make final timeline

In [10]:
shorts = []
for target_person_timeline in people_timeline:
    final_shorts_timeline = make_final_timeline(shorts_timeline,target_person_timeline)
    shorts.append(final_shorts_timeline)

In [11]:
shorts

[([], 0), ([(173.62, 208.74, 1.78, 0.441)], 35.12)]

## save shorts

In [22]:
if not osp.isdir(SHORTS_SAVE_PATH):
    os.makedirs(SHORTS_SAVE_PATH)

for short in shorts:
    if short[1] > 0:
        for ind,(start,end,_,_) in enumerate(short[0]):
            SHORTS_PATH = SHORTS_SAVE_PATH + SHORTS_NAME + str(ind) + '.mp4'
            video = VideoFileClip(VIDEO_PATH).subclip(start,end).fx(vfx.fadein,1).fx(vfx.fadeout,1)
            video.write_videofile(SHORTS_PATH)

chunk:   0%|          | 2/775 [00:00<00:44, 17.37it/s, now=None]

Moviepy - Building video /opt/ml/project/output_dir/shorts/RadioStar_EP00_0.mp4.
MoviePy - Writing audio in RadioStar_EP00_0TEMP_MPY_wvf_snd.mp3


t:   0%|          | 0/1053 [00:00<?, ?it/s, now=None]             

MoviePy - Done.
Moviepy - Writing video /opt/ml/project/output_dir/shorts/RadioStar_EP00_0.mp4



Moviepy - Done !
Moviepy - video ready /opt/ml/project/output_dir/shorts/RadioStar_EP00_0.mp4
